In [58]:
# Import Statements go here
import numpy as np
import numpy.linalg as npla
from scipy import sparse
def pagerank2(E, return_vector = False, max_iters = 1000, tolerance = 1e-6, m = 0.15):
    """compute page rank from dense adjacency matrix
    Inputs:
      E: adjacency matrix with links going from cols to rows.
         E is a matrix of 0s and 1s, where E[i,j] = 1 means 
         that web page (vertex) j has a link to web page i.
      return_vector = False: If True, return the eigenvector as well as the ranking.
      max_iters = 1000: Maximum number of power iterations to do.
      tolerance = 1e-6: Stop when the eigenvector norm changes by less than this.
      m = 0.15: default
      
    Outputs:
      ranking: Permutation giving the ranking, most important first
      vector (only if return_vector is True): Dominant eigenvector of PageRank matrix
    This computes page rank by the following steps:
    1. Add links from any dangling vertices to all vertices.
    2. Scale the columns to sum to 1.
    3. Add a constant matrix to represent jumping at random 15% of the time.
    4. Find the dominant eigenvector with the power method.
    5. Sort the eigenvector to get the rankings.
    
    The function takes input E as a scipy csr_sparse matrix, and then never creates 
    a full matrix or any large matrix other than E.
    """

# HERE ARE ALL THE TASKS YOU NEED TO FIGURE OUT!
#################################################

# 1. Check if E is a square matrix and that its entries are ONLY 1s and 0s, otherwise end this
    assert E.shape[0] == E.shape[1], "Not a square matrix"
    nnz = np.count_nonzero(E.data)
    assert np.max(E) == 1 and np.sum(E) == nnz, "Entries not only 1s and 0s"
    
# 2. Check if E is a sparse matrix. If NOT, then convert it into one.
    #sparse if number non zero is less than 1/3 of the total entries  
    if (type(E) != sparse._csr.csr_matrix):
      E = sparse.csr_matrix(E.data)
      
# 3. Calculate the outdegree 
    #calculate by summing up per row
    n = E.shape[0]
    outdegree = np.array(np.sum(E,0))
    new_array = np.zeros(n)
    for i in range(E.shape[1]):
          new_array[i] = outdegree[0][i]
    outdegree = new_array
# 4. Get set up for the power iteration:
#       create an initial vector (all 1s)
#       make sure you know where outdegree is 0
    n = E.shape[0]
    e = np.ones(n)
    v = e 
    there = np.where(outdegree == 0)
    outdegree[there] = 1
    for iteration in range(max_iters):
        oldv = v
      #Remember: The equation we are trying to solve is: MV = (1−m)(EV+FV) + m*SV,
      #     where:  SV is the average of vector v
      #             EV is matrix E multiplied by the normalized version of vector v
      #             FV is a matrix that accounts for dangling vertices (i.e. where outdegree is 0 in E). 
      #                 Note that if there are no dangling nodes in E, F will always be 0 (easy case).
      
      
      #Part 1: SV -- This last (and easiest) part of the equation essentially is just an 
      # "average" operator on the matrix v. By this I mean that every item in the vector is just 
      # The sum of the vector components divided by the number of nodes, which is an average.
      # You can find SV with a simple and efficient alternative to actually making the dense S matrix and  
      # multiplying it by v to get the same result.
        SV = np.ones(n)
        SV = SV * sum(v)/n
        

      #Part 2: EV  -- Multiply E by normalized v vector
        EV = E @ (v /outdegree)
        
      #Part 3: FV -- In order to avoid having to actually construct the (likely sparse) F matrix,
      # We will take advantage of the unique properties. Essentially we know that for a dangling node,
      # the probability of landing on any other node is evenly split.  This translates to the values
      # at the index of the dangling nodes in FV being one normalized unit of v less than all the other
      # values at non-dangling indicies. You can do this in as little as 3 lines of code and thus 
      # calculate the FV vector without any matrix multiplication. 
        there_np = np.array(there)
        FV = np.ones(n)*np.sum(v[there_np])
        temp_zero = np.zeros(n)
        temp_zero[there_np] = v[there_np] 
        #now remove number v[there] from all positions in FV listed in there
        FV = FV - temp_zero
        #divide FV by (n-1)
        FV = FV /(n-1)
        
        
      #Part 4: Calculate MV per the formula
        MV = (1-m) * (EV + FV) + m * SV
    
    #5. And now you should be back to exactly where we were with pagerank1(), 
    #   so finish this up based on what we did in that function.
        v = MV
        eigval = npla.norm(v)
        v = v / eigval
        if (npla.norm(v - oldv) < tolerance):
              break

    if npla.norm(v - oldv) < tolerance:
        print('Dominant eigenvalue is %f after %d iterations.\n' % (eigval, iteration+1))
    else:
        print('Did not converge to tolerance %e after %d iterations.\n' % (tolerance, max_iters))  
    # finish it up here, just likae in pagerank1()
    assert np.all(v > 0) or np.all(v < 0), 'Error: eigenvector is not all > 0 or < 0'
    vector = np.abs(v)
    
    ranking = np.argsort(vector)[::-1]
    if return_vector: 
      return ranking, vector  
    else:
      return ranking

In [59]:
def pagerank1(E, return_vector = False, max_iters = 1000, tolerance = 1e-6):
    """compute page rank from dense adjacency matrix

    Inputs:
      E: adjacency matrix with links going from cols to rows.
         E is a matrix of 0s and 1s, where E[i,j] = 1 means 
         that web page (vertex) j has a link to web page i.
      return_vector = False: If True, return the eigenvector as well as the ranking.
      max_iters = 1000: Maximum number of power iterations to do.
      tolerance = 1e-6: Stop when the eigenvector norm changes by less than this.
      
    Outputs:
      ranking: Permutation giving the ranking, most important first
      vector (only if return_vector is True): Dominant eigenvector of PageRank matrix

    This computes page rank by the following steps:
    1. Add links from any dangling vertices to all vertices.
    2. Scale the columns to sum to 1.
    3. Add a constant matrix to represent jumping at random 15% of the time.
    4. Find the dominant eigenvector with the power method.
    5. Sort the eigenvector to get the rankings.

    The homework problem asks you to rewrite this code so
    it takes input E as a scipy csr_sparse matrix, and then never creates 
    a full matrix or any large matrix other than E.
    """
    
    if type(E) is not np.ndarray:
        print('Warning, converting input from type', type(E), 'to dense array.')
        E = E.toarray()
                
    nnz = np.count_nonzero(E) # This call for sparse E may be different
    outdegree = np.sum(E, axis=0)  # This call for sparse E may be different
    nrows, n = E.shape

    assert nrows == n, 'E must be square'
    assert np.max(E) == 1 and np.sum(E) == nnz, 'E must contain only zeros and ones'
    
    #  1. Add links from any dangling vertices to all other vertices.
    #     E + F will be the matrix with the added links.

    F = np.zeros((n,n))
    for j in range(n):
        if outdegree[j] == 0:
            F[:,j] = np.ones(n)
            F[j,j] = 0
    
    #  2. Scale the columns to sum to 1 (i.e. normalization of E+F):

    A = (E + F) / np.sum(E + F, axis=0)
    
    #  3. Add a constant matrix to represent jumping at random 15% of the time.

    S = np.ones((n,n)) / n
    m = 0.15
    M = (1 - m) * A + m * S
    
    #  4. Find the dominant eigenvector using the power method.
    #  Start with a vector all of whose entries are equal.

    e = np.ones(n)
    v = e / npla.norm(e)

    for iteration in range(max_iters):
        oldv = v
        
        v = M @ v
        eigval = npla.norm(v)
        v = v / eigval
        
        if npla.norm(v - oldv) < tolerance:
            break
    
    if npla.norm(v - oldv) < tolerance:
        print('Dominant eigenvalue is %f after %d iterations.\n' % (eigval, iteration+1))
    else:
        print('Did not converge to tolerance %e after %d iterations.\n' % (tolerance, max_iters))

    # Check that the eigenvector elements are all the same sign, and make them positive
    assert np.all(v > 0) or np.all(v < 0), 'Error: eigenvector is not all > 0 or < 0'
    vector = np.abs(v)
        
    #  5. Sort the eigenvector and reverse the permutation to get the rankings.
    ranking = np.argsort(vector)[::-1]

    if return_vector:
        return ranking, vector
    else:
        return ranking

# end of pagerank1()


In [60]:
# Non-iterative approach gave us:  
# r = [0 2 3 1]

E = np.load('PageRankEG1.npy')
r, v = pagerank1(E, return_vector = True)
print('r =', r)
print('v =', v)
r, v = pagerank2(E, return_vector = True)
print('r =', r)
print('v =', v)

Dominant eigenvalue is 1.000000 after 19 iterations.

r = [0 2 3 1]
v = [0.69648305 0.26828106 0.54477799 0.38230039]
Dominant eigenvalue is 1.000000 after 19 iterations.

r = [0 2 3 1]
v = [0.69648305 0.26828106 0.54477799 0.38230039]
